<a href="https://www.kaggle.com/code/nicholas33/02-aneurysmnet-cnn-intracranial-training-nb153?scriptVersionId=254354668" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
!pip install monai

# ====================================================
# RSNA INTRACRANIAL ANEURYSM DETECTION - TRAINING PIPELINE
# ====================================================

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pydicom
import nibabel as nib
import cv2
from scipy import ndimage
from monai.networks.nets import BasicUNet
from monai.transforms import Compose, ToTensord
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# ====================================================
# CELL 2: CONFIGURATION
# ====================================================

class Config:
    # Paths
    TRAIN_CSV_PATH = '/kaggle/input/rsna-intracranial-aneurysm-detection/train.csv'
    LOCALIZER_CSV_PATH = '/kaggle/input/rsna-intracranial-aneurysm-detection/train_localizers.csv'
    SERIES_DIR = '/kaggle/input/rsna-intracranial-aneurysm-detection/series/'
    SEGMENTATION_DIR = '/kaggle/input/rsna-intracranial-aneurysm-detection/segmentations/'
    
    # Stage 1: 3D Segmentation
    STAGE1_TARGET_SIZE = (64, 128, 128)  # Smaller for speed
    STAGE1_BATCH_SIZE = 4
    STAGE1_EPOCHS = 5
    STAGE1_LR = 1e-3
    
    # General
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    MIXED_PRECISION = True
    N_FOLDS = 3
    
    # Competition constants
    ID_COL = 'SeriesInstanceUID'
    LABEL_COLS = [
        'Left Infraclinoid Internal Carotid Artery', 'Right Infraclinoid Internal Carotid Artery',
        'Left Supraclinoid Internal Carotid Artery', 'Right Supraclinoid Internal Carotid Artery',
        'Left Middle Cerebral Artery', 'Right Middle Cerebral Artery', 'Anterior Communicating Artery',
        'Left Anterior Cerebral Artery', 'Right Anterior Cerebral Artery',
        'Left Posterior Communicating Artery', 'Right Posterior Communicating Artery',
        'Basilar Tip', 'Other Posterior Circulation', 'Aneurysm Present',
    ]
    TARGET_COL = 'Aneurysm Present'
    
    # Debug settings
    DEBUG_MODE = True
    DEBUG_SAMPLES = 50  # Use small subset for testing

print(f"✅ Configuration loaded - Device: {Config.DEVICE}")


# ====================================================
# CELL 3: SIMPLE DICOM PROCESSOR
# ====================================================

class SimpleDICOMProcessor:
    def __init__(self, target_size=None):
        self.target_size = target_size or Config.STAGE1_TARGET_SIZE
        
    def load_dicom_series(self, series_path):
        """Simple DICOM loading - no complex error handling"""
        try:
            dicom_files = [f for f in os.listdir(series_path) if f.endswith('.dcm')]
            if not dicom_files:
                return np.zeros(self.target_size, dtype=np.float32)
            
            # Load all DICOMs
            pixel_arrays = []
            for f in dicom_files[:50]:  # Limit to 50 files max for speed
                try:
                    ds = pydicom.dcmread(os.path.join(series_path, f), force=True)
                    if hasattr(ds, 'pixel_array'):
                        arr = ds.pixel_array
                        if arr.ndim == 2:  # Standard 2D slice
                            pixel_arrays.append(arr)
                        elif arr.ndim == 3:  # 3D volume - take middle slices
                            mid_start = arr.shape[0] // 4
                            mid_end = 3 * arr.shape[0] // 4
                            for slice_idx in range(mid_start, mid_end, 2):  # Every 2nd slice
                                pixel_arrays.append(arr[slice_idx])
                except:
                    continue
            
            if not pixel_arrays:
                return np.zeros(self.target_size, dtype=np.float32)
            
            # Stack into volume
            volume = np.stack(pixel_arrays, axis=0).astype(np.float32)
            
            # Simple preprocessing
            volume = self.preprocess_volume(volume)
            return volume
            
        except Exception as e:
            print(f"Failed to load {series_path}: {e}")
            return np.zeros(self.target_size, dtype=np.float32)
    
    def preprocess_volume(self, volume):
        """Simple preprocessing"""
        # Normalize
        p1, p99 = np.percentile(volume, [1, 99])
        volume = np.clip(volume, p1, p99)
        volume = (volume - p1) / (p99 - p1 + 1e-8)
        
        # Resize to target
        if volume.shape != self.target_size:
            zoom_factors = [self.target_size[i] / volume.shape[i] for i in range(3)]
            volume = ndimage.zoom(volume, zoom_factors, order=1)
        
        return volume.astype(np.float32)

print("✅ DICOM Processor loaded")

# ====================================================
# CELL 4: DATASET CLASS
# ====================================================

class SimpleSegmentationDataset(Dataset):
    def __init__(self, df, series_dir, processor, mode='train'):
        self.df = df
        self.series_dir = series_dir
        self.processor = processor
        self.mode = mode
        
        # Simple transform
        self.transform = Compose([ToTensord(keys=['volume'])])
        
    def __len__(self):
        return len(self.df)
    
    def load_segmentation_mask(self, series_id, volume_shape):
        """Load real segmentation mask from competition data"""
        seg_path = os.path.join(Config.SEGMENTATION_DIR, f"{series_id}.nii")
        
        try:
            if os.path.exists(seg_path):
                # Load NIfTI segmentation mask
                import nibabel as nib
                nii_img = nib.load(seg_path)
                mask = nii_img.get_fdata().astype(np.float32)
                
                # Resize mask to match volume shape
                if mask.shape != volume_shape:
                    zoom_factors = [volume_shape[i] / mask.shape[i] for i in range(3)]
                    mask = ndimage.zoom(mask, zoom_factors, order=0)  # Nearest neighbor for masks
                
                # Normalize mask values to 0-1
                mask = (mask > 0).astype(np.float32)
                return mask
            else:
                # No segmentation available - create empty mask
                return np.zeros(volume_shape, dtype=np.float32)
                
        except Exception as e:
            print(f"Error loading segmentation for {series_id}: {e}")
            # Fallback: create simple mask if aneurysm present
            has_aneurysm = int(self.df[self.df[Config.ID_COL] == series_id][Config.TARGET_COL].iloc[0])
            if has_aneurysm:
                # Create a rough central region mask as fallback
                mask = np.zeros(volume_shape, dtype=np.float32)
                h, w, d = volume_shape
                mask[h//4:3*h//4, w//4:3*w//4, d//4:3*d//4] = 1.0
                return mask
            else:
                return np.zeros(volume_shape, dtype=np.float32)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        series_id = row[Config.ID_COL]
        series_path = os.path.join(self.series_dir, series_id)
        
        # Load volume
        volume = self.processor.load_dicom_series(series_path)
        
        # Load REAL segmentation mask from competition data
        mask = self.load_segmentation_mask(series_id, volume.shape)
        
        # Get aneurysm presence label
        has_aneurysm = int(row[Config.TARGET_COL])
        
        # Transform
        data_dict = {'volume': volume}
        if self.transform:
            data_dict = self.transform(data_dict)
        
        volume_tensor = data_dict['volume'].unsqueeze(0)  # Add channel dim
        mask_tensor = torch.from_numpy(mask).unsqueeze(0)
        
        return {
            'volume': volume_tensor,
            'mask': mask_tensor,
            'has_aneurysm': torch.tensor(has_aneurysm, dtype=torch.float32),
            'series_id': series_id
        }

print("✅ Dataset class loaded")

# ====================================================
# CELL 5: 3D U-NET MODEL
# ====================================================

class Simple3DSegmentationNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()
        
        # Use MONAI's BasicUNet - simple and proven
        self.backbone = BasicUNet(
            spatial_dims=3,
            in_channels=in_channels,
            out_channels=32,
            features=(32, 64, 128, 256, 32),
            dropout=0.1
        )
        
        # Segmentation head
        self.seg_head = nn.Conv3d(32, out_channels, kernel_size=1)
        
        # Classification head (aneurysm presence)
        self.global_pool = nn.AdaptiveAvgPool3d(1)
        self.classifier = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)
        )
        
    def forward(self, x):
        # Extract features
        features = self.backbone(x)
        
        # Segmentation output
        seg_logits = self.seg_head(features)
        
        # Classification output
        pooled_features = self.global_pool(features).flatten(1)
        cls_logits = self.classifier(pooled_features)
        
        return seg_logits, cls_logits

print("✅ Model architecture loaded")

# ====================================================
# CELL 6: LOSS FUNCTIONS
# ====================================================

class CombinedLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.bce_loss = nn.BCEWithLogitsLoss()
        self.seg_loss = nn.BCEWithLogitsLoss()
        
    def forward(self, seg_logits, cls_logits, seg_targets, cls_targets):
        # Segmentation loss
        seg_loss = self.seg_loss(seg_logits, seg_targets)
        
        # Classification loss
        cls_loss = self.bce_loss(cls_logits.squeeze(), cls_targets)
        
        # Combined loss
        total_loss = seg_loss + 0.5 * cls_loss
        
        return total_loss, seg_loss, cls_loss

print("✅ Loss functions loaded")

# ====================================================
# CELL 7: TRAINING FUNCTIONS
# ====================================================

def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    total_seg_loss = 0
    total_cls_loss = 0
    num_batches = 0
    
    for batch in tqdm(loader, desc="Training"):
        volume = batch['volume'].to(device)
        mask = batch['mask'].to(device)
        has_aneurysm = batch['has_aneurysm'].to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        seg_logits, cls_logits = model(volume)
        
        # Calculate loss
        loss, seg_loss, cls_loss = criterion(seg_logits, cls_logits, mask, has_aneurysm)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_seg_loss += seg_loss.item()
        total_cls_loss += cls_loss.item()
        num_batches += 1
    
    return (total_loss / num_batches, 
            total_seg_loss / num_batches, 
            total_cls_loss / num_batches)

def validate_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    total_seg_loss = 0
    total_cls_loss = 0
    num_batches = 0
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Validating"):
            volume = batch['volume'].to(device)
            mask = batch['mask'].to(device)
            has_aneurysm = batch['has_aneurysm'].to(device)
            
            # Forward pass
            seg_logits, cls_logits = model(volume)
            
            # Calculate loss
            loss, seg_loss, cls_loss = criterion(seg_logits, cls_logits, mask, has_aneurysm)
            
            total_loss += loss.item()
            total_seg_loss += seg_loss.item()
            total_cls_loss += cls_loss.item()
            num_batches += 1
    
    return (total_loss / num_batches, 
            total_seg_loss / num_batches, 
            total_cls_loss / num_batches)

print("✅ Training functions loaded")


# ====================================================
# CELL 8: MAIN TRAINING LOOP
# ====================================================

def main():
    print(f"🚀 STAGE 1: 3D SEGMENTATION FOR REGION LOCALIZATION")
    print(f"Using device: {Config.DEVICE}")
    print(f"Target size: {Config.STAGE1_TARGET_SIZE}")
    
    # Load data
    train_df = pd.read_csv(Config.TRAIN_CSV_PATH)
    
    # Load localizer data (for future use)
    try:
        localizer_df = pd.read_csv(Config.LOCALIZER_CSV_PATH)
        print(f"Loaded localizer data: {len(localizer_df)} entries")
    except:
        localizer_df = None
        print("No localizer data found - continuing without it")
    
    # Debug mode - small subset
    if Config.DEBUG_MODE:
        train_df = train_df.head(Config.DEBUG_SAMPLES)
    print(f"Training samples: {len(train_df)}")
    print(f"Aneurysm cases: {train_df[Config.TARGET_COL].sum()}")
    
    # Simple train/val split
    val_size = len(train_df) // 5
    val_df = train_df[:val_size].reset_index(drop=True)
    train_df = train_df[val_size:].reset_index(drop=True)
    
    print(f"Train: {len(train_df)}, Val: {len(val_df)}")
    
    # Create datasets
    processor = SimpleDICOMProcessor()
    train_dataset = SimpleSegmentationDataset(train_df, Config.SERIES_DIR, processor, 'train')
    val_dataset = SimpleSegmentationDataset(val_df, Config.SERIES_DIR, processor, 'val')
    
    # Create loaders
    train_loader = DataLoader(train_dataset, batch_size=Config.STAGE1_BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=Config.STAGE1_BATCH_SIZE, shuffle=False, num_workers=2)
    
    # Create model
    model = Simple3DSegmentationNet().to(Config.DEVICE)
    
    # Multi-GPU if available
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs")
        model = nn.DataParallel(model)
    
    # Optimizer and loss
    optimizer = optim.AdamW(model.parameters(), lr=Config.STAGE1_LR)
    criterion = CombinedLoss()
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.STAGE1_EPOCHS)
    
    # Training loop
    best_loss = float('inf')
    
    for epoch in range(Config.STAGE1_EPOCHS):
        print(f"\nEpoch {epoch+1}/{Config.STAGE1_EPOCHS}")
        
        # Train
        train_loss, train_seg_loss, train_cls_loss = train_epoch(
            model, train_loader, optimizer, criterion, Config.DEVICE
        )
        
        # Validate
        val_loss, val_seg_loss, val_cls_loss = validate_epoch(
            model, val_loader, criterion, Config.DEVICE
        )
        
        # Step scheduler
        scheduler.step()
        
        print(f"Train - Total: {train_loss:.4f}, Seg: {train_seg_loss:.4f}, Cls: {train_cls_loss:.4f}")
        print(f"Val   - Total: {val_loss:.4f}, Seg: {val_seg_loss:.4f}, Cls: {val_cls_loss:.4f}")
        
        # Save best model
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'epoch': epoch,
                'val_loss': val_loss
            }, 'stage1_segmentation_best.pth')
            print(f"💾 Saved best model (val_loss: {val_loss:.4f})")
    
    print(f"\n✅ Stage 1 complete! Best val loss: {best_loss:.4f}")
    print("📁 Model saved as 'stage1_segmentation_best.pth'")
    
    return model

# ====================================================
# CELL 9: ROI EXTRACTOR FOR STAGE 2 (FUTURE USE)
# ====================================================

class ROIExtractor:
    def __init__(self, roi_size=(224, 224), confidence_threshold=0.5):
        self.roi_size = roi_size
        self.confidence_threshold = confidence_threshold
    
    def extract_rois(self, volume, segmentation_mask):
        """Extract 2D ROI slices from 3D volume using segmentation mask"""
        rois = []
        
        # Find slices with high confidence regions
        for slice_idx in range(volume.shape[0]):
            slice_volume = volume[slice_idx]
            slice_mask = segmentation_mask[slice_idx]
            
            # Check if this slice has potential aneurysm regions
            if np.max(slice_mask) > self.confidence_threshold:
                # Find connected components
                binary_mask = (slice_mask > self.confidence_threshold).astype(np.uint8)
                
                # Find contours
                contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                for contour in contours:
                    # Get bounding box
                    x, y, w, h = cv2.boundingRect(contour)
                    
                    # Expand bounding box
                    margin = max(w, h) // 4
                    x = max(0, x - margin)
                    y = max(0, y - margin)
                    w = min(slice_volume.shape[1] - x, w + 2*margin)
                    h = min(slice_volume.shape[0] - y, h + 2*margin)
                    
                    # Extract ROI
                    roi = slice_volume[y:y+h, x:x+w]
                    
                    # Resize to standard size
                    roi_resized = cv2.resize(roi, self.roi_size)
                    
                    rois.append({
                        'roi': roi_resized,
                        'slice_idx': slice_idx,
                        'bbox': (x, y, w, h),
                        'confidence': np.max(slice_mask[y:y+h, x:x+w])
                    })
        
        return rois

print("✅ ROI Extractor loaded (for Stage 2)")

# ====================================================
# CELL 10: RUN TRAINING
# ====================================================

# Uncomment the line below to start training
# model = main()

print("🎯 Ready to train! Uncomment 'model = main()' in the last cell to start training.")
print("📊 Expected training time: 1-2 hours")
print("💾 Output: stage1_segmentation_best.pth")
